# Binary Classifier (BYOC) Deployment

- Classificador-binario
    - predictor.py: (Flask app for inference)
    - wsgi.py: (Wrapper around predictor)
    - nginx.conf: (Config for nginx front-end)
    - serve: program for container hosting, launches gunicorn server
    - Note that there is no train for pre-trained
- Dockerfile

# Push Docker Image to ECR

In [5]:
%%sh

# Name of algo -> ECR
algorithm_name=aidm-grupo-5-loan-default

cd container

#make serve executable
chmod +x Classificador-binario/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Região
region=$(aws configure get region)
region=${region:-eu-west-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"  #colocar semantic version

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build --network sagemaker -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/sagemaker-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores



Login Succeeded


DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            BuildKit is currently disabled; enable it by removing the DOCKER_BUILDKIT=0
            environment-variable.



Sending build context to Docker daemon  19.97kB
Step 1/11 : FROM python:3.9
 ---> bcd3da597491
Step 2/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> f6d52e02a884
Step 3/11 : RUN python -m pip install --no-cache-dir uv
 ---> Using cache
 ---> a7f681b70818
Step 4/11 : RUN uv pip install --system --no-cache-dir flask gevent gunicorn
 ---> Using cache
 ---> 9d21f5684c2d
Step 5/11 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> e5200f0313bb
Step 6/11 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 405d47a4c039
Step 7/11 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 9cb42ae350fe
Step 8/11 : COPY Classificador-binario /opt/program
 ---> Using cache
 ---> 3d2f803929a5
Step 9/11 : WORKDIR /opt/program
 ---> Using cache
 ---> 0507eee76a6f
Step 10/11 : RUN uv pip install --system --no-cache-dir -r requirements.txt
 ---> Using cache
 ---> 34f244230

# SageMaker Client Setup

- [SageMaker Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model): Model, Endpoint Config, and Endpoint Creation
- [SageMaker RunTime Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint): Endpoint Invocation/Testing

In [6]:
import boto3
from sagemaker import get_execution_role
import pandas as pd
import json
from time import gmtime, strftime

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

#not really used in this use case, use when need to store model artifacts (Ex: MME)
s3_bucket = 'i32419'

role = get_execution_role()

# Model Creation
[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model)

In [7]:
model_name = 'loan-default-model-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

algorithm_name = "aidm-grupo-5-loan-default"
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:latest"

best_model_s3 = "s3://i32419/ai-deployment-monitoring-grupo-5/aidm-loan-default/training-output/grupo-5-aidm-loan-de-260111-1647-001-95c1e479/output/model.tar.gz"

instance_type = 'ml.c5d.18xlarge'

print('Model name:', model_name)
print('Model artifact (best):', best_model_s3)
print('Container image:', image_uri)

primary_container = {
    "Image": image_uri,
    "ModelDataUrl": best_model_s3
}

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer=primary_container
)

print("Model ARN:", create_model_response['ModelArn'])

Model name: loan-default-model-2026-01-11-18-11-02
Model artifact (best): s3://i32419/ai-deployment-monitoring-grupo-5/aidm-loan-default/training-output/grupo-5-aidm-loan-de-260111-1647-001-95c1e479/output/model.tar.gz
Container image: 267567228900.dkr.ecr.eu-west-1.amazonaws.com/aidm-grupo-5-loan-default:latest
Model ARN: arn:aws:sagemaker:eu-west-1:267567228900:model/loan-default-model-2026-01-11-18-11-02


# Endpoint Config Creation

[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config)

In [8]:
endpoint_config_name = 'loan-default-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: loan-default-config2026-01-11-18-11-06
Endpoint config Arn: arn:aws:sagemaker:eu-west-1:267567228900:endpoint-config/loan-default-config2026-01-11-18-11-06


# Endpoint Creation
[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint)

In [9]:
%%time

import time

endpoint_name = 'loan-default-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: loan-default-endpoint-2026-01-11-18-11-09
Endpoint Arn: arn:aws:sagemaker:eu-west-1:267567228900:endpoint/loan-default-endpoint-2026-01-11-18-11-09
Endpoint Status: Creating
Waiting for loan-default-endpoint-2026-01-11-18-11-09 endpoint to be in service...
CPU times: user 23 ms, sys: 25.5 ms, total: 48.5 ms
Wall time: 2min 31s


# Endpoint Invocation

[Documentation](https://boto3.amazonaws.com/v1/documentation/api/1.9.42/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint)

In [13]:
df = pd.read_csv("../data/validation.csv")
X_val = df.drop(columns=["Status"])

sample = X_val.iloc[:5].to_dict(orient="records")  # 5 linhas
request_body = {"instances": sample}           # ou {"input": sample}

payload = json.dumps(request_body)

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

result = json.loads(response["Body"].read().decode("utf-8"))
result

{'predictions': [{'probability': 0.9994614720344543, 'prediction': 1},
  {'probability': 3.246731648687273e-05, 'prediction': 0},
  {'probability': 9.267715358873829e-05, 'prediction': 0},
  {'probability': 1.0978094906022307e-05, 'prediction': 0},
  {'probability': 0.9996709823608398, 'prediction': 1}],
 'threshold': 0.5}

# Delete Endpoint

Before leaving this exercise, it is a good practice to delete the resources created.

In [12]:
# sm_client.delete_endpoint(EndpointName=endpoint_name)
# sm_client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)
# sm_client.delete_model(ModelName=ModelName)